# Simulating Missing Data with `MissMechaGenerator`

This notebook demonstrates how to simulate missing data using the `MissMechaGenerator` from the `missmecha` package.

We use:
- A complete synthetic dataset (`data_num`)
- A single missingness mechanism applied globally across all features
- No per-column customization

For each configuration, we display:
- The generated missing data
- The overall missingness rate
- Little's MCAR test result (to assess if data *may* be MCAR)

## Setup
Import required packages


In [2]:
import numpy as np

from sklearn.model_selection import train_test_split
from missmecha import MissMechaGenerator
from missmecha.analysis import compute_missing_rate, MCARTest

### Generate Complete Data


In [7]:
# Create a synthetic numeric dataset
data_num = np.random.default_rng(1).normal(loc=0.0, scale=1.0, size=(1000, 10))
data_num[:5]

array([[ 0.34558419,  0.82161814,  0.33043708, -1.30315723,  0.90535587,
         0.44637457, -0.53695324,  0.5811181 ,  0.3645724 ,  0.2941325 ],
       [ 0.02842224,  0.54671299, -0.73645409, -0.16290995, -0.48211931,
         0.59884621,  0.03972211, -0.29245675, -0.78190846, -0.25719224],
       [ 0.00814218, -0.27560291,  1.29406381,  1.00672432, -2.71116248,
        -1.88901325, -0.17477209, -0.42219041,  0.213643  ,  0.21732193],
       [ 2.11783876, -1.11202076, -0.37760501,  2.04277161,  0.646703  ,
         0.66306337, -0.51400637, -1.64807517,  0.16746474,  0.10901409],
       [-1.22735205, -0.68322666, -0.07204368, -0.94475162, -0.09826997,
         0.09548303,  0.03558624, -0.50629166,  0.59374807,  0.89116695]])

### Train/Test Split

In [8]:
X_train, X_test = train_test_split(data_num, test_size=0.3, random_state=42)

### Run Simulations Across Mechanism Types

In [10]:
missing_type = "mcar"
mechanism_type_list = [1, 2, 3]
missing_rate_list = [0.3, 0.7]

for mechanism_type in mechanism_type_list:
    for missing_rate in missing_rate_list:
        print(f"Mechanism: {missing_type.upper()}-{mechanism_type} | Missing rate: {missing_rate}")
        
        # Initialize generator
        mecha = MissMechaGenerator(
            mechanism=missing_type,
            mechanism_type=mechanism_type,
            missing_rate=missing_rate,
            seed=42
        )

        # Fit and apply
        X_missing = mecha.fit(X_train)
        X_missing = mecha.transform(X_test)

        # Report missing rate
        compute_missing_rate(X_missing)

        # Run Little's test
        pval = MCARTest(method="little")(X_missing)
        print("-----------------------------------------------------------")

Mechanism: MCAR-1 | Missing rate: 0.3
Global Missing
Overall missing rate: 30.40%
912 / 3000 total values are missing.

Top variables by missing rate:


,n_missing,missing_rate (%),n_unique,dtype,n_total
column,,,,,
col4,98,32.67,202,float64,300
col8,97,32.33,203,float64,300
col7,95,31.67,205,float64,300
col0,95,31.67,205,float64,300
col3,93,31.00,207,float64,300


Method: Little's MCAR Test
Test Statistic p-value: 0.613754
Decision: Fail to reject the null hypothesis (α = 0.05)
→ There is insufficient evidence to reject MCAR.
-----------------------------------------------------------
Mechanism: MCAR-1 | Missing rate: 0.7
Global Missing
Overall missing rate: 69.90%
2097 / 3000 total values are missing.

Top variables by missing rate:


,n_missing,missing_rate (%),n_unique,dtype,n_total
column,,,,,
col7,223,74.33,77,float64,300
col9,215,71.67,85,float64,300
col8,212,70.67,88,float64,300
col5,210,70.00,90,float64,300
col4,209,69.67,91,float64,300


Method: Little's MCAR Test
Test Statistic p-value: 0.266985
Decision: Fail to reject the null hypothesis (α = 0.05)
→ There is insufficient evidence to reject MCAR.
-----------------------------------------------------------
Mechanism: MCAR-2 | Missing rate: 0.3
Global Missing
Overall missing rate: 30.00%
900 / 3000 total values are missing.

Top variables by missing rate:


...

Method: Little's MCAR Test
Test Statistic p-value: 0.346576
Decision: Fail to reject the null hypothesis (α = 0.05)
→ There is insufficient evidence to reject MCAR.
-----------------------------------------------------------
Mechanism: MCAR-3 | Missing rate: 0.3
Global Missing
Overall missing rate: 30.00%
900 / 3000 total values are missing.

Top variables by missing rate:


,n_missing,missing_rate (%),n_unique,dtype,n_total
column,,,,,
col0,90,30.0,210,float64,300
col1,90,30.0,210,float64,300
col2,90,30.0,210,float64,300
col3,90,30.0,210,float64,300
col4,90,30.0,210,float64,300


Method: Little's MCAR Test
Test Statistic p-value: 0.021394
Decision: Reject the null hypothesis (α = 0.05)
→ The data is unlikely to be Missing Completely At Random (MCAR).
-----------------------------------------------------------
Mechanism: MCAR-3 | Missing rate: 0.7
Global Missing
Overall missing rate: 70.00%
2100 / 3000 total values are missing.

Top variables by missing rate:


,n_missing,missing_rate (%),n_unique,dtype,n_total
column,,,,,
col0,210,70.0,90,float64,300
col1,210,70.0,90,float64,300
col2,210,70.0,90,float64,300
col3,210,70.0,90,float64,300
col4,210,70.0,90,float64,300


Method: Little's MCAR Test
Test Statistic p-value: 0.235140
Decision: Fail to reject the null hypothesis (α = 0.05)
→ There is insufficient evidence to reject MCAR.
-----------------------------------------------------------


In [11]:
missing_type = "mar"
mechanism_type_list = [1, 2, 3]
missing_rate_list = [0.3, 0.7]

for mechanism_type in mechanism_type_list:
    for missing_rate in missing_rate_list:
        print(f"Mechanism: {missing_type.upper()}-{mechanism_type} | Missing rate: {missing_rate}")
        
        # Initialize generator
        mecha = MissMechaGenerator(
            mechanism=missing_type,
            mechanism_type=mechanism_type,
            missing_rate=missing_rate,
            seed=42
        )

        # Fit and apply
        X_missing = mecha.fit(X_train)
        X_missing = mecha.transform(X_train)

        # Report missing rate
        compute_missing_rate(X_missing)

        # Run Little's test
        pval = MCARTest(method="little")(X_missing)
        print("-----------------------------------------------------------")

Mechanism: MAR-1 | Missing rate: 0.3
Global Missing
Overall missing rate: 21.19%
1483 / 7000 total values are missing.

Top variables by missing rate:


,n_missing,missing_rate (%),n_unique,dtype,n_total
column,,,,,
col8,225,32.14,475,float64,700
col1,218,31.14,482,float64,700
col7,213,30.43,487,float64,700
col2,213,30.43,487,float64,700
col5,212,30.29,488,float64,700


Method: Little's MCAR Test
Test Statistic p-value: 0.000000
Decision: Reject the null hypothesis (α = 0.05)
→ The data is unlikely to be Missing Completely At Random (MCAR).
-----------------------------------------------------------
Mechanism: MAR-1 | Missing rate: 0.7
Global Missing
Overall missing rate: 48.80%
3416 / 7000 total values are missing.

Top variables by missing rate:


,n_missing,missing_rate (%),n_unique,dtype,n_total
column,,,,,
col8,505,72.14,195,float64,700
col2,496,70.86,204,float64,700
col5,488,69.71,212,float64,700
col3,484,69.14,216,float64,700
col1,483,69.00,217,float64,700


Method: Little's MCAR Test
Test Statistic p-value: 0.000000
Decision: Reject the null hypothesis (α = 0.05)
→ The data is unlikely to be Missing Completely At Random (MCAR).
-----------------------------------------------------------
Mechanism: MAR-2 | Missing rate: 0.3
Global Missing
Overall missing rate: 30.00%
2100 / 7000 total values are missing.

Top variables by missing rate:


...

Method: Little's MCAR Test
Test Statistic p-value: 0.572157
Decision: Fail to reject the null hypothesis (α = 0.05)
→ There is insufficient evidence to reject MCAR.
-----------------------------------------------------------
Mechanism: MAR-3 | Missing rate: 0.3
Global Missing
[MARType3] No label provided. Using synthetic labels instead.
Overall missing rate: 30.00%
2100 / 7000 total values are missing.

Top variables by missing rate:


,n_missing,missing_rate (%),n_unique,dtype,n_total
column,,,,,
col6,222,31.71,478,float64,700
col8,221,31.57,479,float64,700
col3,217,31.00,483,float64,700
col2,216,30.86,484,float64,700
col7,215,30.71,485,float64,700


Method: Little's MCAR Test
Test Statistic p-value: 0.329909
Decision: Fail to reject the null hypothesis (α = 0.05)
→ There is insufficient evidence to reject MCAR.
-----------------------------------------------------------
Mechanism: MAR-3 | Missing rate: 0.7
Global Missing
[MARType3] No label provided. Using synthetic labels instead.
Overall missing rate: 70.00%
4900 / 7000 total values are missing.

Top variables by missing rate:


,n_missing,missing_rate (%),n_unique,dtype,n_total
column,,,,,
col5,500,71.43,200,float64,700
col9,499,71.29,201,float64,700
col7,494,70.57,206,float64,700
col3,491,70.14,209,float64,700
col8,490,70.00,210,float64,700


Method: Little's MCAR Test
Test Statistic p-value: 0.874607
Decision: Fail to reject the null hypothesis (α = 0.05)
→ There is insufficient evidence to reject MCAR.
-----------------------------------------------------------


In [14]:
missing_type = "mnar"
mechanism_type_list = [1, 2, 3]
missing_rate_list = [0.3, 0.7]

for mechanism_type in mechanism_type_list:
    for missing_rate in missing_rate_list:
        print(f"Mechanism: {missing_type.upper()}-{mechanism_type} | Missing rate: {missing_rate}")
        
        # Initialize generator
        mecha = MissMechaGenerator(
            mechanism=missing_type,
            mechanism_type=mechanism_type,
            missing_rate=missing_rate,
            seed=42
        )

        # Fit and apply
        X_missing = mecha.fit(X_train)
        X_missing = mecha.transform(X_train)

        # Report missing rate
        compute_missing_rate(X_missing)

        # Run Little's test
        pval = MCARTest(method="little")(X_missing)
        print("-----------------------------------------------------------")

Mechanism: MNAR-1 | Missing rate: 0.3
Global Missing
self.up_percentile 0.5
Overall missing rate: 50.00%
3500 / 7000 total values are missing.

Top variables by missing rate:


,n_missing,missing_rate (%),n_unique,dtype,n_total
column,,,,,
col0,350,50.0,350,float64,700
col1,350,50.0,350,float64,700
col2,350,50.0,350,float64,700
col3,350,50.0,350,float64,700
col4,350,50.0,350,float64,700


Method: Little's MCAR Test
Test Statistic p-value: 0.395168
Decision: Fail to reject the null hypothesis (α = 0.05)
→ There is insufficient evidence to reject MCAR.
-----------------------------------------------------------
Mechanism: MNAR-1 | Missing rate: 0.7
Global Missing
self.up_percentile 0.5
Overall missing rate: 50.00%
3500 / 7000 total values are missing.

Top variables by missing rate:


,n_missing,missing_rate (%),n_unique,dtype,n_total
column,,,,,
col0,350,50.0,350,float64,700
col1,350,50.0,350,float64,700
col2,350,50.0,350,float64,700
col3,350,50.0,350,float64,700
col4,350,50.0,350,float64,700


Method: Little's MCAR Test
Test Statistic p-value: 0.395168
Decision: Fail to reject the null hypothesis (α = 0.05)
→ There is insufficient evidence to reject MCAR.
-----------------------------------------------------------
Mechanism: MNAR-2 | Missing rate: 0.3
Global Missing
Overall missing rate: 30.90%
2163 / 7000 total values are missing.

Top variables by missing rate:


...

Method: Little's MCAR Test
Test Statistic p-value: 0.374838
Decision: Fail to reject the null hypothesis (α = 0.05)
→ There is insufficient evidence to reject MCAR.
-----------------------------------------------------------
Mechanism: MNAR-3 | Missing rate: 0.3
Global Missing
Overall missing rate: 30.33%
2123 / 7000 total values are missing.

Top variables by missing rate:


,n_missing,missing_rate (%),n_unique,dtype,n_total
column,,,,,
col0,229,32.71,471,float64,700
col4,225,32.14,475,float64,700
col1,217,31.00,483,float64,700
col9,215,30.71,485,float64,700
col8,211,30.14,489,float64,700


Method: Little's MCAR Test
Test Statistic p-value: 0.708031
Decision: Fail to reject the null hypothesis (α = 0.05)
→ There is insufficient evidence to reject MCAR.
-----------------------------------------------------------
Mechanism: MNAR-3 | Missing rate: 0.7
Global Missing
Overall missing rate: 69.57%
4870 / 7000 total values are missing.

Top variables by missing rate:


,n_missing,missing_rate (%),n_unique,dtype,n_total
column,,,,,
col0,493,70.43,207,float64,700
col3,491,70.14,209,float64,700
col7,491,70.14,209,float64,700
col8,491,70.14,209,float64,700
col6,491,70.14,209,float64,700


Method: Little's MCAR Test
Test Statistic p-value: 0.776942
Decision: Fail to reject the null hypothesis (α = 0.05)
→ There is insufficient evidence to reject MCAR.
-----------------------------------------------------------
